Régression linéaire
===================

L'objectif de ce TP est d'effectuer les différents calculs des notions vues en cours sur le modèle
$$Y = X\beta + \varepsilon$$
===========================
dans le cas où $X$ est ou non de plein rang.

On commence par importer les librairies dont on aura besoin pour le TP, à savoir *numpy* pour les outils mathématiques et *sklearn* pour les outils d'apprentissage.

Pour que les *plot* s'affiche dans la page, on ajoute

    %matplotlib inline

In [ ]:
%matplotlib inline
from sklearn import linear_model
import numpy as np
import matplotlib.pyplot as plt

 ## Méthode des moindres carrés ordinaire
 
 Dans un premier temps, nous allons générer $Y$ en fonction de $X$ à partir d'un $\beta$ que l'on se fixe.
 

In [ ]:
n=50
beta=[1,4,1,-7,-2,1,0.7,0.1,3,0.01]
X=np.random.randn(n,10)
eps=np.random.normal(0,1,n)
Y=np.dot(X,beta)+eps
print(Y)

L'objectif est de retrouver $Y$ à partir de l'observation de $X$ et $Y$.

Dans un premier, on affiche sur un graphe $Y$ en fonction d'une coordonnée de $X$.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(Y, X[:,8], alpha=0.5)

ax.set_xlabel(r'$X$', fontsize=20)
ax.set_ylabel(r'$Y$', fontsize=20)
ax.set_title(r'$Y$ en fonction de $X_9\beta_9$')

ax.grid(True)
fig.tight_layout()

plt.show()

On va avoir du mal à prédire $Y$ seulement à partir de $X_9$.

**Calculer l'erreur quadratique moyenne dans cette régression est de $Y$ sur $X_9$ lorsque $\beta_9=3$ est connu.**

On pourra utiliser la méthode

    np.linalg.norm

In [ ]:
#help(np.linalg.norm)

**Puis l'erreur quadratique moyenne de la régression de $Y$ sur $X$ lorsque $\beta$ est connu.**

Comme attendu, l'écart quadratique moyen est inférieur lorsque l'on régresse $Y$ sur toute les colonnes de $X$.
Autrement dit, en utilisant toute l'information que l'on a sur $Y$ (i.e. toute les colonnes de $X$), on fait mieux.

Maintenant, on suppose que $\beta$ est inconnue. 
On veut le retrouver à partir de nos observations de $X$ et $Y$.

**Estimer $\beta$ en appliquant la méthode des moindres carrés.**

On pourra utiliser les méthodes

    np.linalg.inv
    np.linalg.dot
    np.transpose

In [ ]:
#help(np.linalg.inv)
#help(np.linalg.dot)
#help(np.transpose)

La variance de $\varepsilon$ est $1$. À partir de la formule du cours, **donner la variance des $\hat\beta_i$ (conditionnellement à $X$).**

On pourra utiliser 

        np.diag

In [ ]:
#help(np.diag)

**En déduire un intervalle de confiance à 0.95 des $\beta_j$.
Les *vrais* $\beta_j$ sont-ils dans cet intervalle?**

Pour rappel, le quantile d'ordre 0.975 de la loi normale est $\approx$ 1.96

### Cas parcimonieux

On reprend maintenant le même contexte, mais cette fois, des colonnes sont ajoutées à $X$. Autrement dit, l'information qui nous permet d'estimer $X$ (i.e. les colonnes $j$ de $X$ ayant un grand coefficient $\beta_j$) est masquée parmi un grand nombre de colonnnes de $X$.
On note alors $\beta'$ ce nouveau vecteur.

*Pour les modèles avec parcimonie, on notera $k$ le nombre de coordonnées non nulles de $\beta$.*

In [ ]:
d=7000
k=len(beta)
betap=np.concatenate((beta,np.zeros(d-k)))
np.random.shuffle(betap)
print(betap)
Xp=np.random.randn(n,d)
Yp=np.dot(Xp,betap)+eps

**Que renvoie la méthode des moindres carrés?
Commenter**

*Enregistrer votre TP avant de lancer le calcul*

Pour pouvoir retrouver nos coefficients, on va utiliser la méthode LASSO dans un premier temps.

## Méthode LASSO

La méthode de régression LASSO est implémentée dans la librairie *scikitlearn* via la classe

    linear_model.Lasso
   
Nous avons vu qu'il est possible d'ajouter une colonne de $1$ à $X$ pour des données non centrées. Cette classe permet d'automatiser l'ajout de cette colonne, qui est alors appelée *intercept*.
Comme on sait que dans notre modèle, il n'y a pas de colonne de 1 et que cette classe l'ajoute par défaut, on va lui demander de ne pas l'ajouter via l'argument *intercept=False*

*Dans cette classe, le $\lambda$ du cours est appelé* alpha

On utilise la valeur $\lambda$ donnée en cours.

In [ ]:
l=8*np.sqrt(np.log(2*d)/n)
clf = linear_model.Lasso(alpha=l,fit_intercept=False)
clf.fit(Xp, Yp)
hatbetap=clf.coef_

print(betap[betap!=0])
print(betap[hatbetap!=0])
print(hatbetap[betap!=0])

**Commenter. Est-ce que la méthode semble bien fonctionner?**

*Réponse:*

### Vérification des hypothèses
Vérifions si les hypothèses de nos théorèmes sont bien satisfaites.

Pour que l'on puisse appliquer le théorème du cours qui affirme que 
$$\mathbb{E}\frac{1}{n}\|Y-X\hat\beta_{LASSO}\|^2\lesssim k\sigma^2\frac{\log(2d)}{n},$$
où ici $k=len(\beta)=10$;
nous devons vérifier que la matrice $X^tX$ vérifie la condition d'incohérence

$$ \sup \left|\frac{X^tX}{n}-Id\right|\leq \frac{1}{14k}.\hspace{2cm} (1) $$

**$X^tX$ vérifie-t-elle cette hypothèse? Que vaut $\left|\frac{X^tX}{n}-Id\right|_\infty$ pour notre $X$?**

*Ici, $|A|_\infty$ désigne le plus grand coefficient en valeur absolue de la matrice $A$.*

On pourra utiliser les méthodes

    np.amax
    np.identity


In [ ]:
#help(np.linalg.amax)

On a généré les coefficients de notre matrice $X$ (pour un $n$ fixé) à l'aide d'un tirage aléatoire suivant une variable aléatoire de loi $\mathcal{N}(0,1)$.
On note $\mathbb{X}_{i,j}^n$ les coefficients de $\mathbb{X}=X^tX$; chacun de ces coefficients est alors la somme de variables aléatoires indépendantes:
$$\mathbb{X}_{i,j}^n=\sum_{l=1}^n X_{l,i}X_{l,j}.$$

**Pourquoi $\frac{1}{n}\mathbb{X}^n$ converge vers $Id$ lorsque $n\rightarrow\infty$?**
    

*Reponse:*

On peut aussi utiliser le théorème central limite pour estimer la distance entre $\mathbb{X}_{i,j}$ et sa limite quand $n\rightarrow \infty$. On obtient alors un intervalle de confiance pour chaque $\frac{1}{n}\mathbb{X}_{i,j}$ autour de $\mathbf{1}_{\{i=j\}}$.

**Donner un intervalle de confiance de niveau $\alpha=0.05$ de $\frac{1}{n}\mathbb{X}_{i,j}^n$ autour de sa limite.**

*Réponse:*

On peut même calculer $\mathbb{E}\left(\frac{1}{n}\mathbb{X}_{i,j}^n-\mathbf{1}_{i=j}\right)^2$ pour avoir une idée de la valeur moyenne des coefficients de $\frac{1}{n}\mathbb{X}_{i,j}^n-\mathbf{1}_{i=j}$.

**Calculer $\mathbb{E}\left(\frac{1}{n}\mathbb{X}_{i,j}^n-\mathbf{1}_{i=j}\right)^2$**.

*Réponse:*

Cet intervalle de confiance nous indique donc, dans l'idée de vérifier l'équation $(1)$, que chaque entrée de la matrice $\frac{X^tX}{n}-Id$ est plus petite, en valeur absolue, que $\approx 0.14$ - puisque $n=200$, avec probabilité $0.95$.

Et si on regarde la valeur moyenne de $|\frac{1}{n}\mathbb{X}_{i,j}-\mathbf{1}_{i=j}|$, on obtient 
$$
\mathbb{E}\left|\frac{1}{n}\mathbb{X}_{i,j}^n-\mathbf{1}_{i=j}\right|\leq \left(\mathbb{E}\left(\frac{1}{n}\mathbb{X}_{i,j}^n-\mathbf{1}_{i=j}\right)^2\right)^{1/2} = \frac{1}{\sqrt{n}} \approx 0.07
$$

On n'est pas encore à $\frac{1}{14k}\approx 0.007$ qu'il faudrait pour assurer la condition d'incohérence $(1)$.

En fait, les calculs que l'on fait sont valables pour chaque $\frac{1}{n}\mathbb{X}_{i,j}^n$, mais la condition $(1)$ demande à ce que le *maximum* des coefficients de $|\frac{1}{n}\mathbb{X}_{i,j}^n-\mathbf{1}_{i=j}|$ soit majoré, ce qui est plus difficile à obtenir (i.e. plus rare, moins probable) que de demander qu'un des coefficients soit majoré.

Mais ça nous permet de comprendre la chose suivante:

*puisqu'en moyenne $\left|\frac{1}{n}\mathbb{X}_{i,j}^n-\mathbf{1}_{i=j}\right|\approx \frac{1}{\sqrt{n}}$ et que l'on veut satisfaire $(1)$, il faudrait au moins $k\lesssim\sqrt{n}$*

En fait, il existe une autre version de ce théorème qui affirme que l'on peut obtenir la même majoration (avec une grande probabilité):

$$\mathbb{E}\frac{1}{n}\|Y-X\hat\beta_{LASSO}\|^2\lesssim k\sigma^2\frac{\log(2d)}{n}, \hspace{2cm}(2)$$
où ici $k=len(\beta)=10$

dès que
$$k\log(d)\lesssim n$$

lorsque les coefficients de $X$ sont des variables aléatoires gaussiennes i.i.d..

### Erreur quadratique moyenne

On va essayer de vérifier l'équation (2). Pour ça, on se fixe d'abord $d=7000$,
puis on calcule $\frac{1}{n}\|Y-X\hat\beta_{LASSO}\|^2$ pour différentes valeurs de $k$ et de $n$.

Pour ça, on crée plusieurs vecteurs $\beta$, que l'on range dans une grande matrice $\rm B$.

In [ ]:
d=700
n=200
kr=100
m=5

B=np.zeros((d,1))
for k in range(1,kr):
    B=np.concatenate((B,np.concatenate((m*np.ones((k,1)),np.zeros((d-k,1))))),axis=1)
B.shape


Maintenant, on génère $Y$ en fonction de $X$ pour chaque ligne de $\rm B$.

**Générer ces $Y$ en fonction des lignes de $\rm B$ et calculer à chaque fois $\frac{1}{n}\|Y-X\hat\beta_{LASSO}\|^2$, pour $k=1...100$, $n=200$ et $d=700$.
Afficher le résultat sur un graphe et comparer à $\frac{1}{n}\|\beta - \hat\beta_{LASSO}\|^2$.**

**Faire varier $m=5$ et interpréter les différences entre les graphes.**

Réponse:

## Méthode avec pénalisation $|\hspace{0.3cm}|_0$.

Nous avons vu qu'il est possible de pénaliser la minimisation de $\beta \mapsto \|Y-X\beta\|$ par $+\lambda |\beta|_0$ pour chercher les coordonnées non nulles de $\beta$.
Ou de manière équivalente, on peut minimiser $\beta \mapsto \|Y-X\beta\|$ sous la contrainte $|\beta|_0\leq k$.

**Implémenter cette minimisation pour $k=3$**

On pourra utiliser 

    np.inf
    np.argmin
    np.unravel_index
    
**Puis tester pour $d=20,50,100$**

## Méthode Ridge

**Implémenter la méthode Ridge à l'aide des formules du cours.**
**Appliquer pour $X$ et $\beta$ définis par:**

    X=np.random.rand(n,d-1)
    X=np.concatenate((np.transpose([2*X[:,0]]),X),axis=1)
    
    beta=np.concatenate(([1],np.zeros(d-1)))

**On ne retrouve pas le $\beta$ d'origine. Pourquoi?**

*Réponse:*

## Méthode Elastic Net

La méthode Elastic Net combine les deux pénalisations $\| . \|_1$ et $\| . \|_2$.
On minimise alors:

$$ \beta \mapsto \|Y-X\beta\|_2^2 + \alpha\left(\lambda \|\beta\|_1 + (1-\lambda)\|\beta\|_2^2\right) $$
---------------

dans l'espoir de combiner les deux effets des méthodes LASSO et Ridge.

Cette méthode est codée dans la classe

    linear_model.ElasticNet
    
**Implémenter cette méthode pour un jeu de données simulé où:**
+ $X$ est généré par des variables aléatoires de loi $\mathcal{N}(0,1)$ i.i.d. pour ses $d-3$ premières colonnes, puis les 3 dernières sont des combinaisons linéaires de deux des colonnes précédentes.
+ $\varepsilon$ est une suite de v.a.i.i.d. de loi $\mathcal{N}(0,1)$.
+ $\beta=(1,-3,0,...,0,0,3)$.

On demande de retourner $\hat\beta$ par cette méthode.

**Commenter.**

*Réponse:*